In [1]:
import numpy as np
import umbridge
import pandas as pd
import fmpy
from fmpy import *

In [2]:
"""""""""""""""""""""""""""""""""""""""""""""
estimation of parameters (DMEGC_18650_29E)
"""""""""""""""""""""""""""""""""""""""""""""
R_th_c_i = 0.555; #Weil vereinfacht nur eine Schicht betrachtet wird (Annahme: recht genau)
R_th_cell_cell_air = 0.01/(0.1*0.001*85) #Schätzwert da Abstand der Zellen nicht genau bekannt
R_th_cell_cell_con = 0.01/(0.1*0.001*85) #Schätzwert da Abstand der Zellen nicht genau bekannt
R_th_c_o = 0.555 #Weil vereinfacht nur eine Schicht betrachtet wird (Annahme: recht genau)
R_th_cells_case = 1/(0.065 * 2 * np.pi * 0.009*5*7*22)+1/(0.4701*5) #Ungenau, da Fläche von Zellen größer als Fläche vom Case
R_th_case_amb_1 = 2.3524 #right
R_th_case_amb_2 = 0.8574 #top von Interesse wegen Lüftungsschlitzen
R_th_case_amb_3 = 2.3524 #left
R_th_case_amb_4 = 3.0574 #bottom Schätzwert da Übergang Stahl Plexiglas nicht bekannt
R_th_case_amb_5 = 8.5584 #front
R_th_case_amb_6 = 5.9909 #back
#C_th_c_i = 14*22*34.932/2 #recht sicher, da eigentlich bekannte Werte
#C_th_c_o = 14*22*34.932/2
#C_th_case = 110 #uncertain, is calculated by multiplication of literature value for steel (550) (unclear if thats the right material) and mass of module (0.2) (also only approximational value

In [3]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
generating normal distributed independent random variables for monte carlo simulation 
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
parameters = [R_th_c_i, R_th_cell_cell_air, R_th_cell_cell_con, R_th_c_o, R_th_cells_case, R_th_case_amb_1, R_th_case_amb_2, R_th_case_amb_3, R_th_case_amb_4, R_th_case_amb_5, R_th_case_amb_6]
ew_var = []
for parameter in parameters: 
    ew_var.append([parameter, 0.1*parameter])

N = 10000 # mc steps
normal_distributed_samples = [np.random.normal(ew, sd, size=N).tolist() for ew,sd in ew_var]
observation_matrix_normal_distribution = pd.DataFrame({})
for _sample in normal_distributed_samples:
    observation_matrix_normal_distribution = pd.concat([observation_matrix_normal_distribution,pd.Series(_sample)], axis=1)
observation_matrix_normal_distribution = observation_matrix_normal_distribution.T


In [4]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
generating uniformly distributed independent random variables for monte carlo simulation 
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
parameters = [R_th_c_i, R_th_cell_cell_air, R_th_cell_cell_con, R_th_c_o, R_th_cells_case, R_th_case_amb_1, R_th_case_amb_2, R_th_case_amb_3, R_th_case_amb_4, R_th_case_amb_5, R_th_case_amb_6]
uncertainty_intervals = []
for parameter in parameters: 
    uncertainty_intervals.append([parameter-0.1*parameter, parameter+0.1*parameter])

N = 10000 # mc steps
uniformly_distributed_samples = [np.random.uniform(lower, upper, size=N).tolist() for lower,upper in uncertainty_intervals]
observation_matrix_uniform_distribution = pd.DataFrame({})
for _sample in uniformly_distributed_samples:
    observation_matrix_uniform_distribution = pd.concat([observation_matrix_uniform_distribution,pd.Series(_sample)], axis=1)
observation_matrix_uniform_distribution = observation_matrix_uniform_distribution.T

In [5]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
targeting the model of the heat circel from umbridge/simulink
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
#model = umbridge.HTTPModel("http://localhost:4242", "forward")

'""\ntargeting the model of the heat circel from umbridge/simulink\n'

In [ ]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
generating output for ANOVA
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
#mc_values_1 = [model([observation_matrix_normal_distribution[i].tolist()])[0][0] for i in range(0,N)]
#mc_values_2 = [model([observation_matrix_normal_distribution[i].tolist()])[1][0] for i in range(0,N)]

filename = 'PATH/TO/.fmu'

temperature_list = []
for i in range(N):

    output = [
            'Output',
            'Output1',]

    input_df = pd.DataFrame({
        "Input": [observation_matrix_normal_distribution.iloc[0][i]], 
        "Input1": [observation_matrix_normal_distribution.iloc[1][i]], 
        "Input2": [observation_matrix_normal_distribution.iloc[2][i]], 
        "Input3": [observation_matrix_normal_distribution.iloc[3][i]], 
        "Input4": [observation_matrix_normal_distribution.iloc[4][i]], 
        "Input5": [observation_matrix_normal_distribution.iloc[5][i]], 
        "Input6": [observation_matrix_normal_distribution.iloc[6][i]], 
        "Input7": [observation_matrix_normal_distribution.iloc[7][i]], 
        "Input8": [observation_matrix_normal_distribution.iloc[8][i]], 
        "Input9": [observation_matrix_normal_distribution.iloc[9][i]],
        "Input10": [observation_matrix_normal_distribution.iloc[10][i]]
    }, index=[0])

    input_df.insert(0, 'time', 0.0)  # Zeit-Spalte hinzufügen

    input_df.to_csv('PATH/TO/OUTPUT/.csv', index=False)

    input = read_csv('PATH/TO/.csv')

    result = simulate_fmu(filename, input=input, output=output, stop_time=10000)

    temperature_list.append(result[-1])

temperature_1_list = []
temperature_2_list = []

for i in range(N):
    temperature_1_list.append(temperature_list[i][1])
    temperature_2_list.append(temperature_list[i][2])

In [ ]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
monte carlo estimators with uniformly distributed input
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
mc_variance_1 = np.var(temperature_1_list)
mc_variance_2 = np.var(temperature_2_list)
mc_estimator_1 = np.mean(temperature_1_list)
mc_estimator_2 = np.mean(temperature_2_list)
print("Monte Carlo Estimator for first output (normally distributed rv):", mc_estimator_1)
print("Monte Carlo Estimator for second output (normally distributed rv):", mc_estimator_2)
print("Standard deviation of Monte Carlo realisations for first output (normally distributed rv): ", np.sqrt(mc_variance_1))
print("Standard deviation of Monte Carlo realisations for second output (normally distributed rv): ", np.sqrt(mc_variance_2))

Monte Carlo Estimator for first output (normally distributed rv): 306.77596201562835
Monte Carlo Estimator for second output (normally distributed rv): 297.5365143714576
Standard deviation of Monte Carlo realisations for first output (normally distributed rv):  0.2996774052813285
Standard deviation of Monte Carlo realisations for second output (normally distributed rv):  0.192307254072315


In [ ]:
observation_matrix_normal_distribution = observation_matrix_normal_distribution.T
observation_matrix_normal_distribution[11] = temperature_1_list
observation_matrix_normal_distribution[12] = temperature_2_list
observation_matrix_normal_distribution.columns = ['X{}'.format(i) for i in range(0,11)] + ['Y1'] + ['Y2']
observation_matrix_uniform_distribution = observation_matrix_uniform_distribution.T
observation_matrix_uniform_distribution[11] = temperature_1_list
observation_matrix_uniform_distribution[12] = temperature_2_list
observation_matrix_uniform_distribution.columns = ['X{}'.format(i) for i in range(0,11)] + ['Y1'] + ['Y2']


In [ ]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
ANOVA
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
import statsmodels.api as sm
from statsmodels.formula.api import ols

model_1 = ols('Y1 ~ ' + ' + '.join(observation_matrix_normal_distribution.columns[:11]), data=observation_matrix_normal_distribution).fit()
model_2 = ols('Y2 ~ ' + ' + '.join(observation_matrix_normal_distribution.columns[:11]), data=observation_matrix_normal_distribution).fit()

anova_results_1 = sm.stats.anova_lm(model_1, typ=2)
anova_results_2 = sm.stats.anova_lm(model_2, typ=2)

In [114]:
anova_results_1

,sum_sq,df,F,PR(>F)
X0,9.061992,1.0,23006.499491,0.000000e+00
X1,4.482217,1.0,11379.411738,0.000000e+00
X2,4.215084,1.0,10701.215710,0.000000e+00
X3,4.405891,1.0,11185.634078,0.000000e+00
X4,56.242559,1.0,142788.082507,0.000000e+00
X5,0.320988,1.0,814.920925,3.418245e-131
X6,2.813795,1.0,7143.637087,0.000000e+00
X7,0.370953,1.0,941.772688,8.504769e-146
X8,0.178816,1.0,453.975778,3.460673e-83
X9,0.030238,1.0,76.767013,8.244839e-18


In [ ]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
monte carlo simulation ANOVA
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
from SALib.analyze import sobol
from SALib.sample import saltelli

filename = 'C:/Users/felix/OneDrive/Documents/Master Mathematik/Modeling Week/batteriemodell_simscape_2output.fmu'

problem = {
    'num_vars': 11,
    'names': ['X{}'.format(i) for i in range(1, 12)],  # Namen der Input-Variablen
    'bounds': uncertainty_intervals  # Für normalverteilte Daten, 3 Standardabweichungen
}

param_values = saltelli.sample(problem, 10000, calc_second_order=True)

for i in range(len(param_values)):

    output = [
            'Output',
            'Output1',]

    input_df = pd.DataFrame({
        "Input": [param_values[i][0]], 
        "Input1": [param_values[i][1]], 
        "Input2": [param_values[i][2]], 
        "Input3": [param_values[i][3]], 
        "Input4": [param_values[i][4]], 
        "Input5": [param_values[i][5]], 
        "Input6": [param_values[i][6]], 
        "Input7": [param_values[i][7]], 
        "Input8": [param_values[i][8]], 
        "Input9": [param_values[i][9]],
        "Input10": [param_values[i][10]]
    }, index=[0])

    input_df.insert(0, 'time', 0.0)  # Zeit-Spalte hinzufügen

    input_df.to_csv('C:/Users/felix/OneDrive/Documents/Master Mathematik/Modeling Week/data/montecarlo.csv', index=False)

    input = read_csv('C:/Users/felix/OneDrive/Documents/Master Mathematik/Modeling Week/data/montecarlo.csv')

    result = simulate_fmu(filename, input=input, output=output, stop_time=10000)

    temperature_list.append(result[-1])

temperature_1_list = []
temperature_2_list = []

for i in range(len(param_values)):
    temperature_1_list.append(temperature_list[i][1])
    temperature_2_list.append(temperature_list[i][2])

# Führe die Sensitivitätsanalyse durch
Si_1 = sobol.analyze(problem, np.array(temperature_2_list), print_to_console=True)
# Ausgabe der ersten und totalen Sobol-Indizes
print("Erste Sobol-Indizes:", Si_1['S1'])
print("Totale Sobol-Indizes:", Si_1['ST'])

C:\Users\felix\AppData\Local\Temp\ipykernel_11648\2286198355.py:12: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5.1 Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, 16, calc_second_order=True)


           ST   ST_conf
X1   0.797944  0.649705
X2   0.926983  0.643988
X3   1.209508  0.996311
X4   1.060435  0.715476
X5   1.323876  0.910525
X6   2.296980  1.819474
X7   2.460090  1.091843
X8   1.115471  0.904334
X9   1.630431  1.317111
X10  0.922765  0.866414
X11  1.282322  1.086986
           S1   S1_conf
X1   0.326111  0.424268
X2   0.419034  0.402191
X3   0.259135  0.647377
X4   0.376714  0.401913
X5   0.125900  0.613661
X6  -0.443446  1.865967
X7  -0.118650  1.351219
X8   0.138112  0.662062
X9   0.298501  0.628791
X10  0.240951  0.377772
X11  0.385906  0.480840
                  S2   S2_conf
(X1, X2)   -0.827103  0.926163
(X1, X3)   -0.910538  1.323497
(X1, X4)   -0.250545  1.288928
(X1, X5)   -0.196396  1.017731
(X1, X6)   -0.390785  1.955717
(X1, X7)   -0.159221  1.666292
(X1, X8)   -0.620433  1.215831
(X1, X9)   -0.510215  0.834120
(X1, X10)  -0.126021  0.537861
(X1, X11)  -0.406557  0.892011
(X2, X3)    0.039709  0.439298
(X2, X4)   -0.670660  0.696519
(X2, X5)   -0.703411 

C:\Users\felix\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))
